In [35]:
import pandas as pd
df = pd.read_csv('data/Friends_A.tsv', sep='\t')
df.head()

,Unnamed: 0,scene_id,text,character,cAGR,utterance,labels
0,0,01_e01_c01,"[['Ross Geller', "" No!! Okay?! Why does everyo...",Joey,1,"Alright Ross, look. You're feeling a lot of ...",1
1,1,01_e01_c02,"[['Monica Geller', "" Now I'm guessing that he ...",Chandler,1,(imitating the characters) Tuna or egg salad...,1
2,2,01_e01_c04,"[['Ross Geller', "" (squatting and reading the ...",Chandler,0,It's a beautiful thing. I would have to say...,0
3,4,01_e01_c08,"[['Paul', ' Ever since she walked out on me, I...",Monica,1,"What?..... What, you wanna spell it out with...",1
4,5,01_e01_c11,"[['Rachel Green', "" Isn't this amazing? I mean...",Chandler,0,"That is amazing. If can invade Poland, ther...",0


In [39]:
for sent in eval(df['text'][0]):
    print(sent)

['Ross Geller', " No!! Okay?! Why does everyone keep fixating on that? She didn't know, how should I know?"]
['Chandler Bing', ' Sometimes I wish I was a lesbian... (They all stare at him.) Did I say that out loud?']
['Ross Geller', ' I told mom and dad last night, they seemed to take it pretty well.']
['Monica Geller', ' Oh really, so that hysterical phone call I got from a woman at sobbing 3:00 A.M., "I\'ll never have grandchildren, I\'ll never have grandchildren." was what? A wrong number?']
['Ross Geller', ' Sorry.']
['Joey Tribbiani', " Alright Ross, look. You're feeling a lot of pain right now. You're angry. You're hurting. Can I tell you what the answer is?"]
['Joey Tribbiani', " Strip joint! C'mon, you're single! Have some hormones!"]
['Ross Geller', " I don't want to be single, okay? I just... I just- I just wanna be married again!"]


In [43]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


MAX_LEN = 64


df = pd.read_csv('data/Friends_O.tsv', sep='\t')
print(df.shape)
input_ids = [tokenizer.encode(sent, add_special_tokens=True, max_length=MAX_LEN, \
            pad_to_max_length=True) for sent in df['utterance']]


input_ids



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(569, 8)


[[101,
  10303,
  5811,
  1010,
  2298,
  1012,
  2017,
  1005,
  2128,
  3110,
  1037,
  2843,
  1997,
  3255,
  2157,
  2085,
  1012,
  2017,
  1005,
  2128,
  4854,
  1012,
  2017,
  1005,
  2128,
  11878,
  1012,
  2064,
  1045,
  2425,
  2017,
  2054,
  1996,
  3437,
  2003,
  1029,
  6167,
  4101,
  999,
  1039,
  1005,
  12256,
  1010,
  2017,
  1005,
  2128,
  2309,
  999,
  2031,
  2070,
  20752,
  999,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  1006,
  10047,
  16518,
  1996,
  3494,
  1007,
  24799,
  2030,
  8288,
  16521,
  1029,
  5630,
  999,
  1006,
  2128,
  2694,
  1007,
  1051,
  11631,
  1010,
  2016,
  2323,
  2025,
  2022,
  4147,
  2216,
  6471,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  2009,
  1005,
  1055,
  1037,
  3376,
  2518,
  1012,
  1045,
  2052,
  2031,
  2000,
  2360,
  200

In [47]:
print()


['[CLS]', 'alright', 'ross', 'look', 'youre', 'feeling', 'a', 'lot', 'of', 'pain', 'right', 'now', 'youre', 'angry', 'youre', 'hurting', 'can', 'i', 'tell', 'you', 'what', 'the', 'answer', 'is', 'strip', 'joint', 'cmon', 'youre', 'single', 'have', 'some', 'hormones', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [34]:
from transformers import BertConfig, BertModel, BertPreTrainedModel

import torch.nn as nn
import torch.nn.functional as F
import torch


class Baseline_1(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels      = 2
        self.config          = config
        self.bert            = BertModel(config)
        self.personality_cls = nn.Linear(config.hidden_size, self.num_labels)
        self.get_vad         = nn.Linear(config.hidden_size, 3)  # 3 for vad
        self.init_weights()
    
    def forward(self, text, attention_mask):        
        outputs   = self.bert(text, attention_mask)
        embedding = outputs[1]
        logit_p   = self.personality_cls(embedding)
        logit_vad = self.get_vad(embedding)
        return logit_p, logit_vadr
    
model = Baseline_1.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing Baseline_1: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing Baseline_1 from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Baseline_1 from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Baseline_1 were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['personality_cls.weight', 'pe